# How to use tools

:::info Prerequisites

This guide assumes familiarity with the following concepts:

- [Chatbots](/docs/concepts/#messages)
- [Agents](/docs/tutorials/agents)
- [Chat history](/docs/concepts/#chat-history)

:::

This section will cover how to create conversational agents: chatbots that can interact with other systems and APIs using tools.

## Setup

For this guide, we’ll be using an [tool calling agent](/docs/how_to/agent_executor) with a single tool for searching the web. The default will be powered by [Tavily](/docs/integrations/tools/tavily_search), but you can switch it out for any similar tool. The rest of this section will assume you’re using Tavily.

You’ll need to [sign up for an account on the Tavily website](https://tavily.com), and install the following packages:


```{=mdx}
import Npm2Yarn from "@theme/Npm2Yarn";

<Npm2Yarn>
  @langchain/core @langchain/openai langchain
</Npm2Yarn>
```

In [1]:
import { TavilySearchResults } from "@langchain/community/tools/tavily_search";
import { ChatOpenAI } from "@langchain/openai";

const tools = [
  new TavilySearchResults({
    maxResults: 1,
  }),
];

const llm = new ChatOpenAI({
  model: "gpt-3.5-turbo-1106",
  temperature: 0,
});

To make our agent conversational, we must also choose a prompt with a placeholder for our chat history. Here’s an example:


In [2]:
import {
  ChatPromptTemplate,
} from "@langchain/core/prompts";

// Adapted from https://smith.langchain.com/hub/jacob/tool-calling-agent
const prompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    "You are a helpful assistant. You may not need to use tools for every query - the user may just want to chat!",
  ],
  ["placeholder", "{messages}"],
  ["placeholder", "{agent_scratchpad}"],
]);

Great! Now let’s assemble our agent:

```{=mdx}
:::tip
As of `langchain` version `0.2.8`, the `createOpenAIToolsAgent` function now supports [OpenAI-formatted tools](https://api.js.langchain.com/interfaces/langchain_core.language_models_base.ToolDefinition.html).
:::
```


In [3]:
import { AgentExecutor, createToolCallingAgent } from "langchain/agents";

const agent = await createToolCallingAgent({
  llm,
  tools,
  prompt,
});

const agentExecutor = new AgentExecutor({ agent, tools });

## Running the agent

Now that we’ve set up our agent, let’s try interacting with it! It can handle both trivial queries that require no lookup:


In [4]:
import { HumanMessage } from "@langchain/core/messages";

await agentExecutor.invoke({
  messages: [new HumanMessage("I'm Nemo!")],
});

{
  messages: [
    HumanMessage {
      lc_serializable: true,
      lc_kwargs: {
        content: "I'm Nemo!",
        additional_kwargs: {},
        response_metadata: {}
      },
      lc_namespace: [ "langchain_core", "messages" ],
      content: "I'm Nemo!",
      name: undefined,
      additional_kwargs: {},
      response_metadata: {}
    }
  ],
  output: "Hi Nemo! It's great to meet you. How can I assist you today?"
}

Or, it can use of the passed search tool to get up to date information if needed:


In [5]:
await agentExecutor.invoke({
  messages: [
    new HumanMessage(
      "What is the current conservation status of the Great Barrier Reef?"
    ),
  ],
});

{
  messages: [
    HumanMessage {
      lc_serializable: true,
      lc_kwargs: {
        content: "What is the current conservation status of the Great Barrier Reef?",
        additional_kwargs: {},
        response_metadata: {}
      },
      lc_namespace: [ "langchain_core", "messages" ],
      content: "What is the current conservation status of the Great Barrier Reef?",
      name: undefined,
      additional_kwargs: {},
      response_metadata: {}
    }
  ],
  output: "The Great Barrier Reef has recorded its highest amount of coral cover since the Australian Institute"... 688 more characters
}

## Conversational responses

Because our prompt contains a placeholder for chat history messages, our agent can also take previous interactions into account and respond conversationally like a standard chatbot:


In [6]:
import { AIMessage } from "@langchain/core/messages";

await agentExecutor.invoke({
  messages: [
    new HumanMessage("I'm Nemo!"),
    new AIMessage("Hello Nemo! How can I assist you today?"),
    new HumanMessage("What is my name?"),
  ],
});

{
  messages: [
    HumanMessage {
      lc_serializable: true,
      lc_kwargs: {
        content: "I'm Nemo!",
        additional_kwargs: {},
        response_metadata: {}
      },
      lc_namespace: [ "langchain_core", "messages" ],
      content: "I'm Nemo!",
      name: undefined,
      additional_kwargs: {},
      response_metadata: {}
    },
    AIMessage {
      lc_serializable: true,
      lc_kwargs: {
        content: "Hello Nemo! How can I assist you today?",
        tool_calls: [],
        invalid_tool_calls: [],
        additional_kwargs: {},
        response_metadata: {}
      },
      lc_namespace: [ "langchain_core", "messages" ],
      content: "Hello Nemo! How can I assist you today?",
      name: undefined,
      additional_kwargs: {},
      response_metadata: {},
      tool_calls: [],
      invalid_tool_calls: [],
      usage_metadata: undefined
    },
    HumanMessage {
      lc_serializable: true,
      lc_kwargs: {
        content: "What is my name?",
        ad

If preferred, you can also wrap the agent executor in a [`RunnableWithMessageHistory`](/docs/how_to/message_history/) class to internally manage history messages. Let's redeclare it this way:

In [8]:
const agent2 = await createToolCallingAgent({
  llm,
  tools,
  prompt,
});

const agentExecutor2 = new AgentExecutor({ agent: agent2, tools });

Then, because our agent executor has multiple outputs, we also have to set the `outputMessagesKey` property when initializing the wrapper:


In [11]:
import { ChatMessageHistory } from "langchain/stores/message/in_memory";
import { RunnableWithMessageHistory } from "@langchain/core/runnables";

const demoEphemeralChatMessageHistory = new ChatMessageHistory();

const conversationalAgentExecutor = new RunnableWithMessageHistory({
  runnable: agentExecutor2,
  getMessageHistory: (_sessionId) => demoEphemeralChatMessageHistory,
  inputMessagesKey: "messages",
  outputMessagesKey: "output",
});

await conversationalAgentExecutor.invoke(
  { messages: [new HumanMessage("I'm Nemo!")] },
  { configurable: { sessionId: "unused" } }
);

{
  messages: [
    HumanMessage {
      lc_serializable: true,
      lc_kwargs: {
        content: "I'm Nemo!",
        additional_kwargs: {},
        response_metadata: {}
      },
      lc_namespace: [ "langchain_core", "messages" ],
      content: "I'm Nemo!",
      name: undefined,
      additional_kwargs: {},
      response_metadata: {}
    }
  ],
  output: "Hi Nemo! It's great to meet you. How can I assist you today?"
}

In [12]:
await conversationalAgentExecutor.invoke(
  { messages: [new HumanMessage("What is my name?")] },
  { configurable: { sessionId: "unused" } }
);

{
  messages: [
    HumanMessage {
      lc_serializable: true,
      lc_kwargs: {
        content: "I'm Nemo!",
        additional_kwargs: {},
        response_metadata: {}
      },
      lc_namespace: [ "langchain_core", "messages" ],
      content: "I'm Nemo!",
      name: undefined,
      additional_kwargs: {},
      response_metadata: {}
    },
    AIMessage {
      lc_serializable: true,
      lc_kwargs: {
        content: "Hi Nemo! It's great to meet you. How can I assist you today?",
        tool_calls: [],
        invalid_tool_calls: [],
        additional_kwargs: {},
        response_metadata: {}
      },
      lc_namespace: [ "langchain_core", "messages" ],
      content: "Hi Nemo! It's great to meet you. How can I assist you today?",
      name: undefined,
      additional_kwargs: {},
      response_metadata: {},
      tool_calls: [],
      invalid_tool_calls: [],
      usage_metadata: undefined
    },
    HumanMessage {
      lc_serializable: true,
      lc_kwargs: {
     

## Next steps

You've now learned how to create chatbots with tool-use capabilities.

For more, check out the other guides in this section, including [how to add history to your chatbots](/docs/how_to/chatbots_memory).